In [1]:
!pip install tensorflow numpy matplotlib pillow


In [2]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [3]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [14]:
color_to_class = {
    0: (255, 52, 255),  # Flat areas
    1: (28, 230, 255),  # Areas close to water sources
    2: (255, 74, 70),   # Steep areas
    3: (0, 137, 65),    # Dense vegetation areas
    4: (122, 73, 0),    # Occupied areas
    5: (0, 111, 166),   # Restricted areas
    6: (248, 226, 76),  # Agriculture areas
    # 添加其他类别的映射
}

def color_mask_to_categorical(mask, color_to_class, num_classes):
    mask_array = np.array(mask)
    categorical_mask = np.zeros((mask_array.shape[0], mask_array.shape[1]), dtype=int)

    for color, class_id in color_to_class.items():
        matches = np.all(mask_array == np.array(color, dtype=np.uint8), axis=-1)
        categorical_mask[matches] = class_id

    return to_categorical(categorical_mask, num_classes=num_classes)

from PIL import Image
import numpy as np

from PIL import Image
import numpy as np

def load_and_preprocess_image(image_path, mask_path, target_size=(256, 256), num_classes=7):
    try:
        print("Opening image...")
        image = Image.open(image_path)
        print("Resizing image...")
        image = image.resize(target_size)
        print("Converting image to array and normalizing...")
        image = np.array(image) / 255.0

        print("Opening mask...")
        mask = Image.open(mask_path)
        print("Resizing mask...")
        mask = mask.resize(target_size)
        print("Converting mask to categorical...")
        mask = modified_color_mask_to_categorical(mask, color_to_class, num_classes)

        return image, mask
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [5]:
def unet_model(input_size=(256, 256, 3), num_classes=7):
    inputs = Input(input_size)
    # Contracting Path
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)
    # Expansive Path
    u4 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(32, (3, 3), activation='relu', padding='same')(u4)
    u5 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(16, (3, 3), activation='relu', padding='same')(u5)
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c5)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [6]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def remap_mask(mask, remap_dict, default_value=0):
    remapped_mask = np.zeros_like(mask)
    for original_value, new_value in remap_dict.items():
        remapped_mask[mask == original_value] = new_value
    unknown_values = ~np.isin(mask, list(remap_dict.keys()))
    remapped_mask[unknown_values] = default_value  # Assign default value to unknowns
    return remapped_mask

# Example of usage in your data generator
def image_mask_generator(image_dir, mask_dir, batch_size, target_size=(256, 256), num_classes=7):
    image_paths = sorted([os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith('.png')])
    mask_paths = sorted([os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir) if fname.endswith('.png')])
    total = len(image_paths)

    # Example remapping dictionary
    remap_dict = {i: i if i < 7 else 0 for i in range(280)}  # Mapping all other values to 0 or another logic

    while True:
        for start in range(0, total, batch_size):
            end = min(start + batch_size, total)
            batch_images = []
            batch_masks = []
            for img_path, mask_path in zip(image_paths[start:end], mask_paths[start:end]):
                image = Image.open(img_path).resize(target_size)
                mask = Image.open(mask_path).resize(target_size, resample=Image.NEAREST)

                image = np.array(image, dtype=np.float32) / 255.0
                mask = np.array(mask, dtype=np.uint8)

                # Remap mask values
                mask = remap_mask(mask, remap_dict, default_value=0)

                mask = to_categorical(mask, num_classes=num_classes)

                batch_images.append(image)
                batch_masks.append(mask)

            yield np.array(batch_images), np.array(batch_masks)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def test_directory(path):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        path,
        class_mode=None,
        batch_size=8,
        target_size=(256, 256))
    return generator

# 测试图像目录
image_test_gen = test_directory('/content/drive/MyDrive/ML/UNET/model/')
# 测试掩码目录
mask_test_gen = test_directory('/content/drive/MyDrive/ML/UNET/model/')

def image_mask_generator(image_dir, mask_dir, batch_size):
    # 创建两个数据生成器，一个用于图像，一个用于掩码
    image_datagen = ImageDataGenerator(rescale=1./255)
    mask_datagen = ImageDataGenerator(rescale=1./255)

    # 对图像和掩码进行相同的随机变换
    seed = 1
    image_generator = image_datagen.flow_from_directory(
        image_dir,
        class_mode=None,
        batch_size=batch_size,
        target_size=(256, 256),
        seed=seed,
        color_mode='rgb')  # 确保使用RGB模式读取图像

    mask_generator = mask_datagen.flow_from_directory(
        mask_dir,
        class_mode=None,
        batch_size=batch_size,
        target_size=(256, 256),
        seed=seed,
        color_mode='grayscale')  # 如果掩码是单通道的，使用灰度模式

    # 将图像和掩码组合成一个生成器
    train_generator = zip(image_generator, mask_generator)
    return train_generator

Found 560 images belonging to 2 classes.
Found 560 images belonging to 2 classes.


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 定义随机映射函数
def random_map(mask):
    # 将掩码中多余的像素值随机映射到0-6之间
    mask[mask > 6] = np.random.randint(0, 7)
    return mask

def test_directory(path):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        path,
        class_mode=None,
        batch_size=8,
        target_size=(256, 256))
    return generator





In [17]:
image_dir = '/content/drive/MyDrive/ML/UNET/modelimage/'
mask_dir = '/content/drive/MyDrive/ML/UNET/modelmask/'

# 创建模型
model = unet_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
train_gen = image_mask_generator(image_dir, mask_dir, batch_size=8)
model.fit(train_gen, steps_per_epoch=(280 // 8), epochs=10)
model.save('path_to_save_model.h5')  # 保存模型
model.save('path_to_save_model.h5')  # 保存模型

Found 280 images belonging to 1 classes.
Found 280 images belonging to 1 classes.
Epoch 1/10
35/35 [==============================] - 89s 2s/step - loss: 50.0696 - accuracy: 0.3733
Epoch 2/10
35/35 [==============================] - 87s 2s/step - loss: 44452.4727 - accuracy: 0.1677
Epoch 3/10
35/35 [==============================] - 89s 2s/step - loss: 4306579.0000 - accuracy: 0.1704
Epoch 4/10
35/35 [==============================] - 87s 3s/step - loss: 111703024.0000 - accuracy: 0.1273
Epoch 5/10
35/35 [==============================] - 88s 3s/step - loss: 1206575488.0000 - accuracy: 0.1129
Epoch 6/10
35/35 [==============================] - 88s 2s/step - loss: 5793536512.0000 - accuracy: 0.1663
Epoch 7/10
35/35 [==============================] - 88s 3s/step - loss: 13023725568.0000 - accuracy: 0.1706
Epoch 8/10
35/35 [==============================] - 87s 2s/step - loss: 14774170624.0000 - accuracy: 0.1184
Epoch 9/10
35/35 [==============================] - 89s 3s/step - loss: 47785

In [30]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

# 加载模型
model = load_model('path_to_save_model.h5')

# 加载和预处理测试图像
test_image_path = '/content/drive/MyDrive/ML/UNET/testA/4.png'
test_image = Image.open(test_image_path)
test_image = test_image.convert('RGB')  # 确保图像是RGB格式
test_image = test_image.resize((256, 256))  # 调整图像大小
test_image = np.array(test_image) / 255.0  # 归一化图像数据
test_image = np.expand_dims(test_image, axis=0)  # 增加批次维度

# 使用模型预测
predicted_masks = model.predict(test_image)
print(predicted_masks)

1/1 [==============================] - 0s 189ms/step
[[[[0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.99999994]]

  [[0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.99999994]]

  [[0.         0.99999994 0.         ... 0.         0.
    0.        ]
   [0.